In [4]:
import pandas as pd
import sqlite3

/Users/sharan/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/sharan/.matplotlib/matplotlibrc", line #39
  (fname, cnt))


In [5]:
weather = pd.read_csv('../data/forecast_20190828-114302.csv')

In [6]:
# access the parking data scraped and currently stored in SQLITE 
cnx = sqlite3.connect('../data/opendata_parkering.db')
parking = pd.read_sql_query("SELECT * FROM parkering", cnx)

In [7]:
parking.head()

,date,vehicleCount,id,totalSpaces,garageCode
0,2018/03/06 12:35:01,26,1,65,NORREPORT
1,2018/03/06 12:35:01,559,2,1240,SCANDCENTER
2,2018/03/06 12:35:01,508,3,953,BRUUNS
3,2018/03/06 12:35:01,222,4,378,MAGASIN
4,2018/03/06 12:35:01,182,5,210,KALKVAERKSVEJ


In [9]:
# drop Dokk1 because of no data
parking = parking[parking.garageCode != 'DOKK1']

In [10]:
weather['date'] = weather['date'].astype('datetime64[ns]') 
parking['date'] = parking['date'].astype('datetime64[ns]') 

In [11]:
weather.head()

,date,temperature,precipitation,winddirection,windspeed,pressure,symbol
0,2018-03-22 13:00:00,4,0.0,54.2,2.6,1008.6,Cloudy
1,2018-03-22 14:00:00,3,0.0,40.8,2.4,1008.7,Cloudy
2,2018-03-22 15:00:00,4,0.0,32.4,2.2,1008.8,Partly cloudy
3,2018-03-22 16:00:00,4,0.0,41.3,1.9,1008.6,Partly cloudy
4,2018-03-22 17:00:00,4,0.0,81.5,1.8,1008.4,Partly cloudy


In [12]:
parking.head()

,date,vehicleCount,id,totalSpaces,garageCode
0,2018-03-06 12:35:01,26,1,65,NORREPORT
1,2018-03-06 12:35:01,559,2,1240,SCANDCENTER
2,2018-03-06 12:35:01,508,3,953,BRUUNS
3,2018-03-06 12:35:01,222,4,378,MAGASIN
4,2018-03-06 12:35:01,182,5,210,KALKVAERKSVEJ


In [13]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z


In [14]:
import numpy as np

def get_value_from_dict(value):
    count = 0
    if isinstance(value, dict):
        if len(value.keys()) > 0:
            key = list(value.keys())[0]
            return value.get(key)
        else:
            return np.nan
    else: 
        return value

In [15]:
from tqdm import tqdm_notebook
import datetime

row_list = []
for index in tqdm_notebook(range(0, len(parking))):
    row = parking.iloc[index]
    dict1 = {}
    dt = row['date']
    new_dt = datetime.datetime(dt.year, dt.month, dt.day, dt.hour)
    new_df = weather[weather['date'] == new_dt]
    if len(new_df) > 1: 
        new_df = new_df.iloc[0]
    row_list.append(merge_two_dicts(new_df.to_dict(), row.to_dict()))



In [16]:
df = pd.DataFrame(row_list)

In [17]:
import numpy as np
df = df.applymap(lambda x: get_value_from_dict(x))

In [251]:
df[df['date'] == "2018-03-27 13:00:01"]

,date,garageCode,id,precipitation,pressure,symbol,temperature,totalSpaces,vehicleCount,winddirection,windspeed
56524,2018-03-27 13:00:01,NORREPORT,1,0.0,1014.0,Cloudy,4.0,65,14,296.0,1.3
56525,2018-03-27 13:00:01,SCANDCENTER,2,0.0,1014.0,Cloudy,4.0,1240,597,296.0,1.3
56526,2018-03-27 13:00:01,BRUUNS,3,0.0,1014.0,Cloudy,4.0,953,953,296.0,1.3
56527,2018-03-27 13:00:01,MAGASIN,4,0.0,1014.0,Cloudy,4.0,378,350,296.0,1.3
56528,2018-03-27 13:00:01,KALKVAERKSVEJ,5,0.0,1014.0,Cloudy,4.0,210,129,296.0,1.3
56529,2018-03-27 13:00:01,SALLING,6,0.0,1014.0,Cloudy,4.0,700,697,296.0,1.3
56530,2018-03-27 13:00:01,Navitas,8,0.0,1014.0,Cloudy,4.0,449,131,296.0,1.3
56531,2018-03-27 13:00:01,NewBusgadehuset,9,0.0,1014.0,Cloudy,4.0,105,102,296.0,1.3
56532,2018-03-27 13:00:01,Urban Level 1,10,0.0,1014.0,Cloudy,4.0,319,133,296.0,1.3
56533,2018-03-27 13:00:01,Urban Level 2+3,11,0.0,1014.0,Cloudy,4.0,654,263,296.0,1.3


In [18]:
df = df.dropna().reset_index(drop=True)

In [19]:
dummied_df = pd.get_dummies(df)

In [20]:
df.to_csv("merged_data_frame.csv")